In [4]:
# Imports

import os
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision import transforms
from torchvision.transforms import ToPILImage
import json
import numpy as np
import random
import cv2

# Set seed for randomize functions (Ez reproduction of results)
random.seed(100)

In [5]:
# Get path directories for clips and annotations for the TUSimple dataset + ground truth dictionary

root_dir = os.path.dirname(os.getcwd())
annotated_dir = os.path.join(root_dir,'datasets/tusimple/train_set/annotations')
clips_dir = os.path.join(root_dir,'datasets/tusimple/train_set/')
annotated = os.listdir(annotated_dir)

annotations = list()
for gt_file in annotated:
    path = os.path.join(annotated_dir,gt_file)
    json_gt = [json.loads(line) for line in open(path)]
    annotations.append(json_gt)
    
annotations = [a for f in annotations for a in f]

print(len(annotations))
print(annotations[2]['lanes'])

3626
[[-2, 570, 554, 538, 522, 505, 489, 473, 456, 440, 424, 407, 391, 375, 359, 342, 326, 310, 293, 277, 261, 244, 228, 212, 196, 179, 163, 147, 130, 114, 98, 81, 65, 49, 33, 16, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], [-2, -2, 601, 606, 611, 616, 621, 625, 630, 635, 640, 645, 650, 655, 660, 665, 670, 675, 680, 684, 689, 694, 699, 704, 709, 714, 719, 724, 729, 734, 738, 743, 748, 753, 758, 763, 768, 773, 778, 783, 788, 793, 797, 802, 807, 812, 817, 822], [-2, -2, 516, 482, 449, 415, 382, 348, 314, 281, 247, 214, 180, 147, 113, 80, 46, 13, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], [-2, -2, 647, 673, 698, 724, 750, 775, 801, 826, 852, 877, 903, 928, 954, 980, 1005, 1031, 1056, 1082, 1107, 1133, 1158, 1184, 1210, 1235, 1261, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2]]


In [66]:
masks = np.zeros((720,1280,3),dtype= np.int32)
masks[0]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [43]:
# Generate segmentation masks functionality/ Useful for resizing ground truth masks NOTE: dims = (H,W,C)
img_path = annotations[21]['raw_file']
image = cv2.imread(os.path.join(clips_dir, img_path))

# print(image)
def generate_seg_mask(ground_truth: dict, image, image_dims = (720,1280,3)):
    masks = np.zeros_like(image[:,:,0])
    nolane_token = -2 
    h_vals = ground_truth['h_samples']
    lanes = ground_truth['lanes']
    lane_val = 255
    # NOT WORKING AS IT SHOULD 
    lane_markings_list = []
    for lane in lanes:
        x_coords = []
        y_coords = []
        for i in range(0,len(lane)):             
            if lane[i] != nolane_token:
                x_coords.append(lane[i])
                y_coords.append(h_vals[i])
                lane_markings = list(zip(x_coords, y_coords))
                # print(lane_markings)
        lane_markings_list.append(lane_markings)          
        # lane_pix = np.array(lane_pix)                      
        # height,width,chan = zip(*lane_pix)
        # print(lane_pix)
        # np.put(masks,lane_pix,255)
    for z in lane_markings_list:
        print(z)
        for x,y in z:
            masks[y,x] = 1
    lane_markings_img = cv2.bitwise_and(image, image, mask=masks)
    return lane_markings_img  

# print(annotations[0])
masked_image = generate_seg_mask(annotations[21], image)
# masks.shape
cv2.imshow('Original Image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()        
cv2.imshow('Masked Image', masked_image)
cv2.waitKey(0)
cv2.destroyAllWindows()    

[(730, 280), (705, 290), (681, 300), (661, 310), (640, 320), (619, 330), (598, 340), (577, 350), (556, 360), (535, 370), (514, 380), (493, 390), (472, 400), (451, 410), (433, 420), (416, 430), (398, 440), (381, 450), (363, 460), (346, 470), (328, 480), (311, 490), (293, 500), (276, 510), (258, 520), (241, 530), (223, 540), (205, 550), (188, 560), (170, 570), (153, 580), (135, 590), (118, 600), (100, 610), (83, 620), (65, 630), (48, 640), (30, 650), (12, 660)]
[(884, 260), (848, 270), (831, 280), (816, 290), (816, 300), (815, 310), (815, 320), (814, 330), (814, 340), (814, 350), (817, 360), (820, 370), (823, 380), (826, 390), (829, 400), (832, 410), (835, 420), (838, 430), (842, 440), (845, 450), (848, 460), (851, 470), (854, 480), (857, 490), (860, 500), (863, 510), (866, 520), (869, 530), (872, 540), (875, 550), (878, 560), (881, 570), (885, 580), (888, 590), (891, 600), (894, 610), (897, 620), (900, 630), (903, 640), (906, 650), (909, 660), (912, 670), (915, 680), (918, 690), (921, 7

In [4]:
# Loading/Resizing image test scenario

img_path = annotations[0]['raw_file']
train_transforms = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(size=(640,640))])
image = cv2.imread(os.path.join(clips_dir, img_path))
print(image.shape)
image_tensor = train_transforms(image)
print(image_tensor.shape)



# EZ convert from normalized tensor to np.array (0,255) scale for RGB images
convert = transforms.Compose([transforms.ToPILImage()])
array = np.array(convert(image_tensor))
print(array.shape)


(720, 1280, 3)
torch.Size([3, 640, 640])
(640, 640, 3)


In [5]:
# TuSimple Dataset loader and pre-processing class
# Full Size: Train(3626 clips/ 20 frames per clip/ 20th only is annotated), Test(2782 clips/ 20 frames per clip/ 20th only annotated)
# Link: https://github.com/TuSimple/tusimple-benchmark/tree/master/doc/lane_detection
class TuSimple(Dataset):  
    def __init__(self, train_annotations : list, train_img_dir: str, resize_to : tuple , subset_size = 0.2, image_size = (1280,720), val_size = 0.15):
        self.image_size = image_size
        self.resize = resize_to
        self.val_size = val_size
        self.subset = subset_size
        self.train_dir = train_img_dir
        self.complete_gt = train_annotations
        self.complete_size = len(train_annotations)
        self.train_dataset, self.train_gt = self.generate_dataset()
        
    def __len__(self):
        if len(self.train_dataset) == len(self.train_gt):
            return len(self.train_gt)
        else:
            return "Dataset generation failure: Size of training images does not match the existing ground truths."
    
    def __getitem__(self, idx):
        if len(self.train_dataset) == len(self.train_gt):
            img_tensor = self.train_dataset[idx]
            img_gt = self.train_gt[idx]
            return img_tensor, img_gt
        else:
            return "The dataset hasn't been constructed properly. Generate again!"
    
    # Returns original image size for the dataset    
    def get_image_size(self):
        return self.image_size
        
    # Partition dataset according to input subset size and dynamically generate the train/val splits
    def generate_dataset(self):
        train_set = []
        
        complete_idx = [idx for idx in range(0, self.complete_size + 1)]
        target_samples = int(self.complete_size * self.subset)
        # val_samples = int(len(target_samples) * self.val_size)
        shuffled = random.sample(complete_idx,len(complete_idx))
        
        # Pick n (target samples no) idx from the shuffled dataset
        dataset_idxs = [shuffled[idx] for idx in range(0, target_samples)]
        train_gt = [self.complete_gt[idx] for idx in dataset_idxs]
        
        # Load images, resize inputs, transform to tensors and generate dataset (or subset)
        for gt in train_gt:
            img_path = gt['raw_file']
            train_transforms = transforms.Compose([transforms.ToTensor(),
                                                   transforms.Resize(size = self.resize)])
            image = cv2.imread(os.path.join(self.train_dir, img_path))
            img_tensor = train_transforms(image)
            train_set.append(img_tensor)
        
        return train_set, train_gt   
        

In [6]:
dataset = TuSimple(train_annotations = annotations, train_img_dir = clips_dir, resize_to = (640,640), subset_size = 0.05)